In [180]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [181]:
from puzzles import load

In [182]:
import numpy as np

---

# Решаем про бантики и обертки для подарков

https://adventofcode.com/2015/day/2

In [183]:
A = np.array([l.split("x") for l in load(2).strip().split("\n")]).astype(int)

In [184]:
S = A * np.roll(A, 1, axis=1)
S

array([[ 72,  33, 264],
       [247,  65,  95],
       [ 27,   9, 243],
       ...,
       [ 45,  36, 180],
       [408, 600, 425],
       [154,  84,  66]])

In [185]:
(S.sum(axis=1) * 2 + S.min(axis=1)).sum()

1588178

---

In [186]:
As = np.sort(A, axis=1)[:, :2]
As

array([[ 3, 11],
       [ 5, 13],
       [ 1,  9],
       ...,
       [ 3, 12],
       [17, 24],
       [ 6, 11]])

In [187]:
(2 * As.sum(axis=1) + A.prod(axis=1)).sum()

3783758

---

# Решаем задачу про [последовательность](https://en.wikipedia.org/wiki/Look-and-say_sequence)

https://adventofcode.com/2015/day/10

(run length encoding)

In [192]:
from itertools import groupby

from tqdm import tqdm

import itertools, functools

In [193]:
p = load(10).strip()

for _ in tqdm(range(40)):
    p = "".join([f"{len(list(group))}{item}" for item, group in groupby(p)])

len(p)

100%|███████████████████████████████████████████| 40/40 [00:01<00:00, 38.80it/s]


360154

In [194]:
p = load(10).strip()

for _ in tqdm(range(50)):
    p = "".join([f"{len(list(group))}{item}" for item, group in groupby(p)])

len(p)

100%|███████████████████████████████████████████| 50/50 [00:12<00:00,  4.14it/s]


5103798

----

# Решаем задачку из мира jRPG

https://adventofcode.com/2015/day/21

In [195]:
load(21).strip().split("\n")

['Hit Points: 103', 'Damage: 9', 'Armor: 2']

---

In [197]:
from collections import namedtuple

In [198]:
Weapon = namedtuple("Weapon", "damage gold")

In [199]:
w = (10, 500)
w[0]

10

In [200]:
w = Weapon(damage=10, gold=500)

In [201]:
w.damage

10

In [203]:
from dataclasses import dataclass

In [117]:
@dataclass
class Character:
    max_hp: int
    hp: int
    damage: int
    armor: int

---

In [202]:
WEAPONS = [
    [8, 4],
    [10, 5],
    [25, 6],
    [40, 7],
    [74, 8],
]

ARMOR = [
    [0, 0],
    [13, 1],
    [31, 2],
    [53, 3],
    [75, 4],
    [102, 5],
]

RINGS = [
    [0, 0, 0],
    [0, 0, 0],
    [25, 1, 0],
    [50, 2, 0],
    [100, 3, 0],
    [20, 0, 1],
    [40, 0, 2],
    [80, 0, 3],
]

In [204]:
Character = namedtuple("Character", "hp damage armor")

In [205]:
def build_player(wid, aid, rids):
    gw, damage = WEAPONS[wid]
    ga, armor = ARMOR[aid]
    gold = ga + gw

    for r in rids:
        g, dmg_add, arm_add = RINGS[r]
        gold += g
        damage += dmg_add
        armor += arm_add

    return gold, Character(hp=100, damage=damage, armor=armor)

In [206]:
build_player(-1, -1, [-4, -1])

(356, Character(hp=100, damage=11, armor=8))

In [207]:
from itertools import product

In [209]:
def is_battle_successful(player, boss):
    player_hp = player.hp
    boss_hp = boss.hp

    while True:
        boss_hp -= max(player.damage - boss.armor, 1)
        if boss_hp <= 0:
            return True

        player_hp -= max(boss.damage - player.armor, 1)
        if player_hp <= 0:
            return False

In [210]:
min_gold_spent = 1e9
max_gold_spent = -1

for wid, aid, rid1, rid2 in product(
    range(len(WEAPONS)), range(len(ARMOR)), range(len(RINGS)), range(len(RINGS)),
):
    if rid1 == rid2:
        continue

    gold, player = build_player(wid, aid, [rid1, rid2])
    success = is_battle_successful(player, boss)

    if success and gold < min_gold_spent:
        min_gold_spent = gold

    if not success and gold > max_gold_spent:
        max_gold_spent = gold

min_gold_spent, max_gold_spent

(121, 201)

---

# Решаем ["жизнь" Конвея](https://en.wikipedia.org/wiki/Conway's_Game_of_Life)

https://adventofcode.com/2015/day/18

In [155]:
A = np.array([[".#".index(c) for c in x] for x in load(18).strip().split("\n")])

In [157]:
def rule(k):
    if k[1][1] == 1:
        return int(k.sum() - 1 in [2, 3])
    return int(k.sum() == 3)

In [171]:
from tqdm import tqdm

In [175]:
A = np.array([[".#".index(c) for c in x] for x in load(18).strip().split("\n")])
B = np.zeros_like(A)

for _ in tqdm(range(100)):
    Ap = np.pad(A, 1)
    for i in range(1, len(Ap) - 1):
        for j in range(1, len(Ap[0]) - 1):
            B[i - 1, j - 1] = rule(Ap[i - 1 : i + 2, j - 1 : j + 2])
    A = B

100%|█████████████████████████████████████████| 100/100 [00:39<00:00,  2.55it/s]


In [176]:
A.sum()

814